In [1]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())


C:\Users\pushkar\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16861204565460524075
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1499457126
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6074495061906963370
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [2]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing import image
from PIL import Image

def plot_histogram(image):
    hist,bins = np.histogram(image.flatten(),256,[0,256])

    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()

    plt.plot(cdf_normalized, color = 'b')
    plt.hist(image.flatten(),256,[0,256], color = 'r')
    plt.xlim([0,256])
    plt.legend(('cdf','histogram'), loc = 'upper left')
    plt.show()

def equalize_histogram(img, grid_size):
    clahe = cv2.createCLAHE(tileGridSize=(grid_size,grid_size))
    cl1 = clahe.apply(img)
    return cl1

# img = cv2.imread('./images/train/c0/img_34.jpg', 0)
# equalized_img = equalize_histogram(img,16)
# plt.imshow(equalized_img)
# plt.show()
# plot_histogram(equalized_img)

# ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
# plt.imshow(equalized_img)
# plt.show()
# plot_histogram(equalized_img)

def equalize_histogram(local_img, grid_size):
#    local_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(tileGridSize=(grid_size,grid_size))
    equalized_img = clahe.apply(local_img)
    return equalized_img

def hisEqulColor(img):
    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    print ("channels: ", len(channels))
    cv2.equalizeHist(channels[0],channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)
    return img
    
img = image.load_img('./images/train/c5/img_56.jpg', target_size=(224, 224), grayscale=True)
print("original image shape: ", img)
img = np.array(img)
# eq_img = equalize_histogram(img,16)
# plt.imshow(eq_img)
# plt.show()
# x = image.img_to_array(img)
# print(x.shape)
plt.imshow(img)
plt.show()

kernel = np.ones((8,8),np.uint8)
gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
plt.imshow(gradient)
plt.show()

tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
plt.imshow(tophat)
plt.show()

blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
plt.imshow(blackhat)
plt.show()

dilation = cv2.dilate(img,kernel,iterations = 1)
plt.imshow(dilation)
plt.show()

ret, thresh = cv2.threshold(img, 127, 255, 0)
im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour = cv2.drawContours(img, contours, -1, (0,255,0), 3)
plt.imshow(contour)
plt.show()


original image shape:  <PIL.Image.Image image mode=L size=224x224 at 0x1B186F856D8>


Using TensorFlow backend.


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [3]:
from sklearn.metrics import confusion_matrix
from keras.models import load_model

import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.models import load_model
import datetime, random, pickle
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob
import os
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.preprocessing import image
from tqdm import tqdm
from PIL import ImageFile
import cv2
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, ActivityRegularization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.utils import plot_model
from keras import regularizers
import sys
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

def loadImages(path):
    data = load_files(path)
    files = data['filenames']
    targets = data['target']
    target_names = data['target_names']
    return files, targets, target_names

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    #equalize histogram
    #equalized_img = equalize_histogram(img_array, 16)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)


def paths_to_tensor(img_paths):
    print (img_paths)
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

# ## Load the Data

print ("Loading Images...")
path = "images/train"
files, targets, target_names = loadImages(path)
# predict_files = np.array(glob("images/test/*"))[1:10]
print('Number of Categories: ', len(target_names))
print('Categories: ', target_names)

# Split the original training sets into training & testing sets
train_files, test_files, train_targets, test_targets = train_test_split(files, targets, test_size=0.20, random_state=40)

print(train_files.shape, test_files.shape, train_targets.shape, test_targets.shape)
print(len(test_files))




Loading Images...
Number of Categories:  10
Categories:  ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
(15200,) (3800,) (15200,) (3800,)
3800


In [4]:
print ("Creating image tensors")

train_tensors = paths_to_tensor(train_files).astype('float32') / 255
test_tensors = paths_to_tensor(test_files).astype('float32') / 255

train_targets_onehot = np_utils.to_categorical(np.array(train_targets), 10)
test_targets_onehot = np_utils.to_categorical(np.array(test_targets), 10)

print("Size of train tensors: " + str(train_tensors.shape))
print("Size of test tensors: " + str(test_tensors.shape))
print("Size of test targets: " + str(test_targets.shape))


Creating image tensors
['images/train\\c3\\img_24663.jpg' 'images/train\\c8\\img_98810.jpg'
 'images/train\\c9\\img_67390.jpg' ... 'images/train\\c7\\img_31727.jpg'
 'images/train\\c7\\img_82756.jpg' 'images/train\\c5\\img_21995.jpg']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15200/15200 [01:43<00:00, 147.08it/s]


['images/train\\c5\\img_68264.jpg' 'images/train\\c6\\img_69335.jpg'
 'images/train\\c2\\img_12280.jpg' ... 'images/train\\c8\\img_6916.jpg'
 'images/train\\c6\\img_21610.jpg' 'images/train\\c5\\img_46343.jpg']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3800/3800 [00:30<00:00, 125.78it/s]


Size of train tensors: (15200, 224, 224, 3)
Size of test tensors: (3800, 224, 224, 3)
Size of test targets: (3800,)


In [31]:
import glob

def predict_distraction(model):
    print("Evaluating...")
    scores = model.evaluate(test_tensors, test_targets_onehot, verbose=0)
    print("Evaluation %s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    return

model_files = glob.glob('./saved_models/create_model*.hdf5')
for m in model_files:
    if ('grayscale' not in m):
        print(m)
        model = load_model(m)
        predict_distraction(model)
    #p = model.predict(test_tensors)



./saved_models\create_model102018-03-1684_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 79.50%
./saved_models\create_model112018-03-1879_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 40.71%
./saved_models\create_model112018-03-1976_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 86.97%
./saved_models\create_model12018-03-1446_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 19.21%
./saved_models\create_model12018-03-1551_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 21.26%
./saved_models\create_model12018-03-1714_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 31.00%
./saved_models\create_model122018-03-1921_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 73.71%
./saved_models\create_model132018-03-2055_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 27.18%
./saved_models\create_model142018-03-1959_model.best.from_scratch.hdf5
Evaluating...
Evaluation acc: 77.18%
./saved_models\create_model1520

In [ ]:
# print(test_targets_onehot)
#print(np.argmax(p, axis=0))
# print(np.argmax(p[1,1:10]))

In [ ]:
import glob
from keras.models import load_model

from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

#model_files = glob.glob('./saved_models/*.hdf5')
# for m in model_files:
#     print(m)
#     model = load_model(m)
#     #plot_model(model, to_file='./model_pics/' + m + '.png')
#     SVG(model_to_dot(model).create(prog='dot', format='svg'))